# wps_full_rvic

#### wps_full_rvic is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) and [convolution](https://rvic.readthedocs.io/en/latest/user-guide/convolution/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

url = 'http://localhost:5002/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
osprey.full_rvic?

Signature:
osprey.full_rvic(
    version=True,
    np=1,
    loglevel='INFO',
    params_config_file=None,
    params_config_dict=None,
    convolve_config_file=None,
    convolve_config_dict=None,
)
Docstring:
Run full RVIC process combining Parameters and Convolution modules

Parameters
----------
params_config_file : ComplexData:mimetype:`text/cfg`
    Path to input configuration file or input dictionary
params_config_dict : string
    Dictionary containing input configuration for Parameters process
convolve_config_file : ComplexData:mimetype:`text/cfg`
    Path to input configuration file for Convolution process
convolve_config_dict : string
    Dictionary containing input configuration for Convolution process
version : boolean
    Return RVIC version string
np : integer
    Number of processors used to run job
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Outp

#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# run full_rvic with local configuration files
output_config_files = osprey.full_rvic(
    params_config_file = resource_filename("tests", "/data/configs/parameters_local.cfg"),
    convolve_config_file = resource_filename("tests", "/data/configs/convolve_opendap.cfg")
)

In [4]:
# run parameters with configuration dictionaries
params_config_dict = {
    "OPTIONS": {
        "CASEID": "sample",
        "GRIDID": "COLUMBIA",
    },
    "POUR_POINTS": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/RVIC/sample_pour.txt",
    },
    "UH_BOX": {
        "FILE_NAME": "tests/data/samples/uhbox.csv",
    },
    "ROUTING": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_flow_parameters.nc",
    },
    "DOMAIN": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_routing_domain.nc",
    },
}

convolve_config_dict = {
    "OPTIONS": {
        "CASEID": "sample",
        "CALENDAR": "standard",
        "RUN_STARTDATE": "2012-12-01-00",
        "STOP_DATE": "2012-12-31",
    },
    "DOMAIN": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample_routing_domain.nc",
    },
    "PARAM_FILE": {
        "FILE_NAME": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/sample.rvic.prm.COLUMBIA.20180516.nc",
    },
    "INPUT_FORCINGS": {
        "DATL_PATH": "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/RVIC/",
        "DATL_FILE": "columbia_vicset2.nc",
    },
}

output_config_dicts = osprey.full_rvic(
    params_config_dict = params_config_dict,
    convolve_config_dict = convolve_config_dict,
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [5]:
params_config = read_config("tests/data/configs/parameters_local.cfg")
convolve_config = read_config("tests/data/configs/convolve_opendap.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC history file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": params_config["POUR_POINTS"]["FILE_NAME"].split("/")[-1],
    "RvicUHFile": params_config["UH_BOX"]["FILE_NAME"].split("/")[-1],
    "RvicFdrFile": params_config["ROUTING"]["FILE_NAME"].split("/")[-1],
    "RvicDomainFile": convolve_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}


In [6]:
def test_full_rvic_output(output, expected):
    with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
        output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

        # Metadata of the final output of RVIC parameters + convolution 
        metadata = {
            "title": output_data.title,
            "year": output_data.history.split()[5],
            "month": output_data.history.split()[2],
            "day": output_data.history.split()[3],
            "RvicPourPointsFile": output_data.RvicPourPointsFile,
            "RvicUHFile": output_data.RvicUHFile,
            "RvicFdrFile": output_data.RvicFdrFile,
            "RvicDomainFile": output_data.RvicDomainFile,
        }
        assert metadata == expected

In [7]:
test_full_rvic_output(output_config_files, expected)
test_full_rvic_output(output_config_dicts, expected)